In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader as pdr
import yfinance as yf
from mpl_finance import candlestick_ohlc
import matplotlib.dates as mdates
from arch import arch_model
import datetime
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import QuantLib as ql
plt.style.use('ggplot')
plt.rcParams.update({    "font.family": "sans-serif",
    "font.sans-serif": "Helvetica",})
yf.pdr_override()

In [7]:
symbol = 'SLV'
start = '2010-01-01'
# get current date
end = pd.to_datetime('today')
# get data

ticker_object = yf.Ticker(symbol)
data_raw = ticker_object.history(start=start, end=end)[['Open', 'High', 'Low', 'Close', 'Volume']]
data_raw['$r_t$'] = data_raw['Close'].apply(np.log).diff() * 100
data_raw = data_raw.dropna()
data_raw.index = data_raw.index.tz_localize(None)

In [8]:
from pyrugarch import *

from rpy2.robjects.packages import importr
import rpy2.robjects as robjects
from rpy2.robjects import numpy2ri
from rpy2.robjects import r, pandas2ri
from rpy2.robjects.conversion import localconverter

numpy2ri.activate()
pandas2ri.activate()

batch_baseline = Batch(name='baseline2', ModelClass=RugarchModel, dataset=data_raw)
batch_baseline.setExportDirectories(modelFitDataPath='./model_output/model_data/', modelFitS4Path='./model_output/summaries/')

batch_baseline.addModels(models=[RugarchModel(
                                id='SLV_norm',
                                endog=f'$r_t$',
                                mean_exog=None, 
                                variance_exog=None,
                                garch_model='sGARCH', 
                                arma_order=(0,0),
                                garch_order=(1,1), 
                                distribution_model='norm'),

                        RugarchModel(
                                id='SLV_std',
                                endog=f'$r_t$',
                                mean_exog=None, 
                                variance_exog=None,
                                garch_model='sGARCH', 
                                arma_order=(0,0),
                                garch_order=(1,1), 
                                distribution_model='std')
                            ],
                    verbose=False)
batch_baseline.batchFit(rugarch_fit=True, verbose=True)
batch_baseline.export()
batch_baseline.modelFitTable()

22/02/2024 18:18:10 | baseline2 | Starting batch fit on 2 models
22/02/2024 18:18:11 | baseline2 | Completed batch fit on 2 models


,SLV_norm,SLV_std
mu,-0.0223,0.0145
,(0.0268),(0.0232)
omega,0.0329,0.0175***
,(0.0229),(0.0047)
alpha1,0.0568**,0.0327***
,(0.0238),(0.0033)
beta1,0.9342***,0.9635***
,(0.0276),(0.0005)
shape,nan,4.1464
AIC,3.8319,3.7343
